In [1]:
import logging
import numpy as np
from docplex.mp.model_reader import ModelReader

import numpy as np
from docplex.mp.basic import Expr
from docplex.mp.model import Model
from docplex.mp.solution import SolveSolution
from docplex.util.status import JobSolveStatus
import cplex.callbacks as cpx_cb
#from docplex.mp.callbacks.cb_mixin import ModelCallbackMixin
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer



# Sample 1D ising model with constraints

In [2]:
from docplex.mp.model import Model
opt_model = Model(name="MIP Model")

Ny= 10
## Ny is the number of spin sites

num_flip = 1
### we have to check the constraint


J_i_j = 1
h_i = 0.05

y={}
for i in range(0, Ny):
    y[i]= opt_model.binary_var(name=f"y_{i}")


"Note that: original variables y[t] are binary variables {0,1}, "
"To convert it into the spin variables {+1, -1}, we have to transform s[t] = (-1+(2*y[t]))"
" when y[t]= 0 then s[t] = -1 and when y[t]= 1 then s[t] = 1 "

objective = opt_model.linear_expr()
## objective is the hamiltonian/energy value we want to minimize


for t in range(0, Ny-1):
    objective += (-1+(2*y[t]))*J_i_j*(-1+(2*y[t+1]))
    
for q in range(0, Ny):
    objective += (-1+(2*y[q]))* h_i
    

opt_model.minimize(objective)

## add constraint

opt_model.add_constraint(opt_model.sum((-1+(2*y[r])) for r in range(0, Ny)) >= num_flip , "constraint") 

#### Print the optimization model

opt_model.print_information() 

print(opt_model.prettyprint())               



Model: MIP Model
 - number of variables: 10
   - binary=10, integer=0, continuous=0
 - number of constraints: 1
   - linear=1
 - parameters: defaults
 - objective: minimize quadratic
 - problem type is: MIQP
// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;
dvar bool y_3;
dvar bool y_4;
dvar bool y_5;
dvar bool y_6;
dvar bool y_7;
dvar bool y_8;
dvar bool y_9;

minimize
 - 1.900000 y_0 - 3.900000 y_1 - 3.900000 y_2 - 3.900000 y_3 - 3.900000 y_4
 - 3.900000 y_5 - 3.900000 y_6 - 3.900000 y_7 - 3.900000 y_8 - 1.900000 y_9 [
 4 y_0*y_1 + 4 y_1*y_2 + 4 y_2*y_3 + 4 y_3*y_4 + 4 y_4*y_5 + 4 y_5*y_6
 + 4 y_6*y_7 + 4 y_7*y_8 + 4 y_8*y_9 ] + 8.500000;
 
subject to {
 constraint:
  2 y_0 + 2 y_1 + 2 y_2 + 2 y_3 + 2 y_4 + 2 y_5 + 2 y_6 + 2 y_7 + 2 y_8
  + 2 y_9 -10 >= 1;

}
None


In [3]:
result = opt_model.solve(log_output=True) #(log_output=self.solver_config.cplex_log)
y_value=[]
for l in range(0, Ny):
    y_value.append(result.get_value(f"y_{l}"))
    print(f"y_{l} =" , result.get_value(f"y_{l}")  )
    

print("Binary Variables" , y_value, "Objective value", result.objective_value )


Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 9.500000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve added 18 rows and 9 columns.
MIP Presolve modified 2 coefficients.
Reduced MIP has 19 rows, 19 columns, and 46 nonzeros.
Reduced MIP has 19 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 9 rows and 0 columns.
Reduced MIP has 10 rows, 19 columns, and 37 nonzeros.
Reduced MIP has 19 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 s

# Sample 2D Ising Model with constraints

In [4]:
from docplex.mp.model import Model
opt_model = Model(name="MIP Model")

Ny= 20
## Ny is the number of spin sites in the X and Y direction so total sites = Ny x Ny (Ny square)
num_flip = 0.5
### we have to check the constraint

J_i_j = 1.0
h_i = 0.5

x={}
for i in range(0, Ny):
    for j in range(0, Ny):
        x[i,j]= opt_model.binary_var(name=f"x_{i}_{j}")


"Note that: original variables x[tx, ty] are binary variables {0,1}, "
"To convert it into the spin variables {+1, -1}, we have to transform s[t] = (-1+(2*x[tx, ty]))"
" when y[t]= 0 then s[t] = -1 and when y[t]= 1 then s[t] = 1 "

objective = opt_model.linear_expr()

## objective is the hamiltonian/energy value we want to minimize

for tx in range(0, Ny-1):
    objective -= opt_model.sum((-1+(2*x[tx, ty]))*J_i_j*(-1+(2*x[tx+1, ty])) for ty in range(0, Ny))

for ty in range(0, Ny-1):
    objective -= opt_model.sum((-1+(2*x[tx, ty]))*J_i_j*(-1+(2*x[tx, ty+1])) for tx in range(0, Ny))
    
objective += opt_model.sum(((-1+(2*x[qx,qy]))* h_i) for qx in range(0, Ny) for qy in range(0, Ny))
    

opt_model.minimize(objective)


opt_model.add_constraint((opt_model.sum((-1+(2*x[rx, ry])) for rx in range(0, Ny) for ry in range(0, Ny))/(Ny*Ny)) >= num_flip , "constraint") 


opt_model.print_information() 

print(opt_model.prettyprint())               


Model: MIP Model
 - number of variables: 400
   - binary=400, integer=0, continuous=0
 - number of constraints: 1
   - linear=1
 - parameters: defaults
 - objective: minimize quadratic
 - problem type is: MIQP
// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0_0;
dvar bool x_0_1;
dvar bool x_0_2;
dvar bool x_0_3;
dvar bool x_0_4;
dvar bool x_0_5;
dvar bool x_0_6;
dvar bool x_0_7;
dvar bool x_0_8;
dvar bool x_0_9;
dvar bool x_0_10;
dvar bool x_0_11;
dvar bool x_0_12;
dvar bool x_0_13;
dvar bool x_0_14;
dvar bool x_0_15;
dvar bool x_0_16;
dvar bool x_0_17;
dvar bool x_0_18;
dvar bool x_0_19;
dvar bool x_1_0;
dvar bool x_1_1;
dvar bool x_1_2;
dvar bool x_1_3;
dvar bool x_1_4;
dvar bool x_1_5;
dvar bool x_1_6;
dvar bool x_1_7;
dvar bool x_1_8;
dvar bool x_1_9;
dvar bool x_1_10;
dvar bool x_1_11;
dvar bool x_1_12;
dvar bool x_1_13;
dvar bool x_1_14;
dvar bool x_1_15;
dvar bool x_1_16;
dvar bool x_1_17;
dvar bool x_1_18;
dvar bool x_1_

In [5]:
result = opt_model.solve(log_output=True) #(log_output=self.solver_config.cplex_log)
x_value=[]

for i in range(0, Ny):
    for j in range(0, Ny):
        x_value.append(int(result.get_value(f"x_{i}_{j}")))
        print(f"x_{i}_{j} =" , int(result.get_value(f"x_{i}_{j}")))
    

print("Binary Variables" , x_value, "Objective value", result.objective_value )


Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Found incumbent of value -560.000000 after 0.00 sec. (0.03 ticks)
Tried aggregator 1 time.
MIP Presolve added 1520 rows and 760 columns.
Reduced MIP has 1521 rows, 1160 columns, and 3440 nonzeros.
Reduced MIP has 1160 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.39 ticks)
Probing time = 0.00 sec. (0.85 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1521 rows, 1160 columns, and 3440 nonzeros.
Reduced MIP has 1160 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (4.38 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 0.00 sec. (0.85 ticks)
Clique table members: 1520.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.02 sec. (28.37 ticks)

        Nodes 